<a href="https://colab.research.google.com/github/FatherOfLove/258_DL/blob/master/unGraded5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
data = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = data.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
#scaling
X_train, X_test = X_train / 255.0, X_test / 255.0

In [0]:
# Add  channels dimension
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

In [5]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
def CNN_model():
  from keras.models import Sequential
  from keras.layers import BatchNormalization
  model = Sequential([
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape = (32,32,1)),#Dropout(0.75),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.75),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.75),
         BatchNormalization(axis=3),
         MaxPool2D(pool_size=(2,2)),
         #Dropout(0.5),

         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         MaxPool2D(pool_size=(2,2)), 
         #Dropout(0.5),

         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),#Dropout(0.25),
         BatchNormalization(axis=3),
         MaxPool2D(pool_size=(2,2)), 
         #Dropout(0.5),   

         Flatten(),   
         Dense(256, activation = "relu"),
         #Dropout(0.75),   
         (Dense(10, activation = "softmax"))])


  model.compile(optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) , 
                loss = "categorical_crossentropy", metrics=["accuracy"])

  return model

Using TensorFlow backend.


In [0]:
class BiuldModel(Model):
  def __init__(self):
    super(BiuldModel, self).__init__()
    self.conv_layer = Conv2D(32,3,activation='relu')
    self.flatten_layer = Flatten()
    self.dense1 = Dense(128,activation='relu')
    self.dense2 = Dense(10)

  def call(self, X):
    X = self.conv_layer(X)
    X = self.flatten_layer(X)
    X = self.dense1(X)
    return self.dense2(X)

In [7]:
model = BiuldModel()
model

In [0]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = keras.optimizers.Adam()

In [0]:
train_loss = keras.metrics.Mean(name='train_loss')
train_accuracy =keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = keras.metrics.Mean(name='test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
def train_step(imags, labels):
  with tf.GradientTape() as tape:  
    preds_train = model(imags, training=True)
    loss_train = loss(labels, preds_train)
  grads = tape.gradient(loss_train, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss_train)
  train_accuracy(labels, preds_train)

In [0]:
def test_step(imags, labels): 
  preds_test = model(imags, training=False)
  loss_test = loss(labels, preds_test)

  test_loss(loss_test)
  test_accuracy(labels, preds_test)

In [0]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)

test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [14]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  for imgs, labels in train:
    train_step(imgs, labels)

  for test_imgs, test_labels in test:
    test_step(test_imgs, test_labels)

  str_temp = 'Epoch Num {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(str_temp.format(epoch + 1,train_loss.result(),train_accuracy.result() * 100,
                        test_loss.result(),test_accuracy.result() * 100))

KeyboardInterrupt: ignored

In [0]:
|